In [7]:
#basic imports
import pandas as pd
import numpy as np
import math
#ml stuff
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense
from keras.models import Model
from keras.models import Sequential
#metrics + sklearn
from scipy import spatial
from hdbscan import HDBSCAN
from sklearn import metrics
from sklearn import decomposition
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import numpy as np 
#data visualization
import plotly.graph_objs as go
import plotly
import hiplot as hip

C:\Users\madke\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [8]:
#aware merge will strip down to 120
aware = pd.read_csv('/users/madke/downloads/aware_personality_8-4.csv', index_col = 0)
awaregb = aware.groupby('participantID')


aware['localDate'] =pd.to_datetime(aware.localDate)
aware = aware.loc[:, aware.isna().mean() < .40]

sequences = []
for i in list(set(aware['participantID'])):
    unit = awaregb.get_group(i)
    unit = unit.sort_values(by='localDate')
    unit = unit.reset_index(drop = True)
    sequences.append(unit.reset_index(drop = False))


imp_seq = []
for i in sequences:
    needs_fixed = i.iloc[:,2:(len(i.columns)-10)]
    fixed = needs_fixed.fillna(needs_fixed.mean())
    fixed = fixed.fillna(0)
    new = pd.concat([i.iloc[:,0:2], fixed, i.iloc[:, (len(i.columns)-10):(len(i.columns))]], axis = 1)
    imp_seq.append(new)
  

In [9]:
dep_vars = []
indep_vars = []
for i in imp_seq:
    dep = i.iloc[:, (len(i.columns)-10):(len(i.columns))]
    dep = dep.iloc[:1, :]
    dep_vars.append(np.array(dep))
    indep = i.iloc[:, 0:len(i.columns)-10].drop(['participantID', 'localDate'], axis = 1)
    indep_vars.append(indep)
#here is padding the values for nns (same size 90th percentile)
pad_imp = []
for i in indep_vars:
    if len(i['index']) >= 21:
        pad_imp.append(np.array(i.iloc[0:21,:]))
    else:
        count = 21 - len(i['index'])
        for z in range(0,count):
            i.loc[len(i)] = 0
        pad_imp.append(np.array(i))
    

In [31]:
# split a  sequence into samples
#https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/
def split_sequence(sequence, n_steps = 3):
    X, y = list(), list()
    for i in range(sequence.shape[0]):
# find the end of this pattern
        end_ix = i + n_steps
    # check if we are beyond the sequence
        if end_ix > sequence.shape[0] -1:
            break
            # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix, :], sequence[end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)
 


In [120]:
X_ls = []
y_ls = []
for i in pad_imp:
    X, y = split_sequence(i)
    X_ls.append(X)
    y_ls.append(y)

In [121]:
X_ls[1].shape

(18, 3, 385)

In [124]:
dep_vars_ls = y_ls
indep_vars_ls =X_ls
indep_vars_ls = np.where(np.isnan(indep_vars_ls),0,indep_vars_ls)


y = np.vstack(dep_vars_ls)

# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
y = scaler.fit_transform(y)

#y = y.reshape(120,18,385)
y = np.where(np.isnan(y),0,y)

# normalize the dataset
X = np.array(indep_vars_ls)
scaled = np.vstack(X)

print(y.shape)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaled.reshape(2160,3,385)
X = scaled
X = np.where(np.isnan(X),0,X)
#[samples, timesteps, features]

(2160, 385)


In [98]:
y.shape

(2160, 385)

In [129]:
learn = 0.01
opt = tf.optimizers.Adam(lr= learn, amsgrad = True)

loss_fctn = 'mean_squared_error'
activ_h = 'tanh'  #selu, tanh
KI_h = 'glorot_uniform'
activ_d = 'sigmoid'
n_steps, n_features = X.shape[1], X.shape[2]


model = Sequential()
model.add(LSTM(50, activation=activ_h, kernel_initializer = KI_h, input_shape=(n_steps, n_features)))
model.add(Dense(385, input_shape=(n_steps, n_features), activation=activ_d))
model.compile(loss=loss_fctn, optimizer=opt,  metrics=['acc', 'mae', 'msle', 'mse'])

In [130]:
model.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_18 (LSTM)               (None, 50)                87200     
_________________________________________________________________
dense_14 (Dense)             (None, 385)               19635     
Total params: 106,835
Trainable params: 106,835
Non-trainable params: 0
_________________________________________________________________


In [131]:
history = model.fit(X, y, epochs=200, verbose=1, validation_split = 0.2)

Train on 1728 samples, validate on 432 samples
Epoch 1/200
1728/1728 [==============================] - 1s 361us/step - loss: 0.0482 - acc: 0.2703 - mae: 0.1408 - msle: 0.0310 - mse: 0.0482 - val_loss: 0.0206 - val_acc: 0.4259 - val_mae: 0.0647 - val_msle: 0.0113 - val_mse: 0.0206
Epoch 2/200
1728/1728 [==============================] - 0s 148us/step - loss: 0.0170 - acc: 0.4288 - mae: 0.0621 - msle: 0.0096 - mse: 0.0170 - val_loss: 0.0195 - val_acc: 0.4167 - val_mae: 0.0634 - val_msle: 0.0108 - val_mse: 0.0195
Epoch 3/200
1728/1728 [==============================] - 0s 136us/step - loss: 0.0166 - acc: 0.3993 - mae: 0.0601 - msle: 0.0093 - mse: 0.0166 - val_loss: 0.0196 - val_acc: 0.5000 - val_mae: 0.0610 - val_msle: 0.0106 - val_mse: 0.0196
Epoch 4/200
1728/1728 [==============================] - 0s 132us/step - loss: 0.0164 - acc: 0.4606 - mae: 0.0579 - msle: 0.0092 - mse: 0.0164 - val_loss: 0.0195 - val_acc: 0.4907 - val_mae: 0.0594 - val_msle: 0.0106 - val_mse: 0.0195
Epoch 5/200
1

1728/1728 [==============================] - 0s 147us/step - loss: 0.0149 - acc: 0.3721 - mae: 0.0518 - msle: 0.0083 - mse: 0.0149 - val_loss: 0.0192 - val_acc: 0.4167 - val_mae: 0.0574 - val_msle: 0.0106 - val_mse: 0.0192
Epoch 36/200
1728/1728 [==============================] - 0s 128us/step - loss: 0.0150 - acc: 0.3802 - mae: 0.0517 - msle: 0.0084 - mse: 0.0150 - val_loss: 0.0191 - val_acc: 0.4537 - val_mae: 0.0582 - val_msle: 0.0105 - val_mse: 0.0191
Epoch 37/200
1728/1728 [==============================] - 0s 136us/step - loss: 0.0149 - acc: 0.3756 - mae: 0.0519 - msle: 0.0084 - mse: 0.0149 - val_loss: 0.0194 - val_acc: 0.4491 - val_mae: 0.0582 - val_msle: 0.0106 - val_mse: 0.0194
Epoch 38/200
1728/1728 [==============================] - 0s 130us/step - loss: 0.0150 - acc: 0.3981 - mae: 0.0519 - msle: 0.0084 - mse: 0.0150 - val_loss: 0.0193 - val_acc: 0.4236 - val_mae: 0.0575 - val_msle: 0.0105 - val_mse: 0.0193
Epoch 39/200
1728/1728 [==============================] - 0s 144us/st

Epoch 70/200
1728/1728 [==============================] - 0s 117us/step - loss: 0.0148 - acc: 0.3906 - mae: 0.0511 - msle: 0.0083 - mse: 0.0148 - val_loss: 0.0195 - val_acc: 0.4630 - val_mae: 0.0580 - val_msle: 0.0107 - val_mse: 0.0195
Epoch 71/200
1728/1728 [==============================] - 0s 132us/step - loss: 0.0148 - acc: 0.4028 - mae: 0.0510 - msle: 0.0083 - mse: 0.0148 - val_loss: 0.0194 - val_acc: 0.1991 - val_mae: 0.0579 - val_msle: 0.0107 - val_mse: 0.0194
Epoch 72/200
1728/1728 [==============================] - 0s 131us/step - loss: 0.0148 - acc: 0.3750 - mae: 0.0506 - msle: 0.0082 - mse: 0.0148 - val_loss: 0.0195 - val_acc: 0.2199 - val_mae: 0.0585 - val_msle: 0.0106 - val_mse: 0.0195
Epoch 73/200
1728/1728 [==============================] - 0s 132us/step - loss: 0.0148 - acc: 0.3628 - mae: 0.0509 - msle: 0.0082 - mse: 0.0148 - val_loss: 0.0196 - val_acc: 0.2847 - val_mae: 0.0584 - val_msle: 0.0107 - val_mse: 0.0196
Epoch 74/200
1728/1728 [==============================] 

Epoch 105/200
1728/1728 [==============================] - 0s 147us/step - loss: 0.0147 - acc: 0.3495 - mae: 0.0507 - msle: 0.0082 - mse: 0.0147 - val_loss: 0.0200 - val_acc: 0.2106 - val_mae: 0.0588 - val_msle: 0.0109 - val_mse: 0.0200
Epoch 106/200
1728/1728 [==============================] - 0s 171us/step - loss: 0.0148 - acc: 0.4080 - mae: 0.0504 - msle: 0.0082 - mse: 0.0148 - val_loss: 0.0197 - val_acc: 0.4259 - val_mae: 0.0580 - val_msle: 0.0107 - val_mse: 0.0197
Epoch 107/200
1728/1728 [==============================] - 0s 119us/step - loss: 0.0147 - acc: 0.3542 - mae: 0.0506 - msle: 0.0082 - mse: 0.0147 - val_loss: 0.0196 - val_acc: 0.4769 - val_mae: 0.0577 - val_msle: 0.0107 - val_mse: 0.0196
Epoch 108/200
1728/1728 [==============================] - 0s 132us/step - loss: 0.0148 - acc: 0.4462 - mae: 0.0504 - msle: 0.0082 - mse: 0.0148 - val_loss: 0.0197 - val_acc: 0.4236 - val_mae: 0.0579 - val_msle: 0.0108 - val_mse: 0.0197
Epoch 109/200
1728/1728 [===========================

1728/1728 [==============================] - 0s 128us/step - loss: 0.0147 - acc: 0.4508 - mae: 0.0503 - msle: 0.0082 - mse: 0.0147 - val_loss: 0.0199 - val_acc: 0.5602 - val_mae: 0.0576 - val_msle: 0.0109 - val_mse: 0.0199
Epoch 140/200
1728/1728 [==============================] - 0s 132us/step - loss: 0.0147 - acc: 0.4132 - mae: 0.0502 - msle: 0.0082 - mse: 0.0147 - val_loss: 0.0200 - val_acc: 0.4583 - val_mae: 0.0591 - val_msle: 0.0109 - val_mse: 0.0200
Epoch 141/200
1728/1728 [==============================] - 0s 122us/step - loss: 0.0146 - acc: 0.4317 - mae: 0.0501 - msle: 0.0082 - mse: 0.0146 - val_loss: 0.0198 - val_acc: 0.4954 - val_mae: 0.0587 - val_msle: 0.0109 - val_mse: 0.0198
Epoch 142/200
1728/1728 [==============================] - 0s 146us/step - loss: 0.0147 - acc: 0.4560 - mae: 0.0504 - msle: 0.0082 - mse: 0.0147 - val_loss: 0.0198 - val_acc: 0.2176 - val_mae: 0.0581 - val_msle: 0.0108 - val_mse: 0.0198
Epoch 143/200
1728/1728 [==============================] - 0s 161u

Epoch 174/200
1728/1728 [==============================] - 0s 181us/step - loss: 0.0147 - acc: 0.4491 - mae: 0.0501 - msle: 0.0082 - mse: 0.0147 - val_loss: 0.0202 - val_acc: 0.5255 - val_mae: 0.0592 - val_msle: 0.0110 - val_mse: 0.0202
Epoch 175/200
1728/1728 [==============================] - 0s 152us/step - loss: 0.0147 - acc: 0.4271 - mae: 0.0500 - msle: 0.0082 - mse: 0.0147 - val_loss: 0.0199 - val_acc: 0.4352 - val_mae: 0.0581 - val_msle: 0.0109 - val_mse: 0.0199
Epoch 176/200
1728/1728 [==============================] - 0s 133us/step - loss: 0.0146 - acc: 0.4676 - mae: 0.0499 - msle: 0.0082 - mse: 0.0146 - val_loss: 0.0200 - val_acc: 0.4213 - val_mae: 0.0586 - val_msle: 0.0109 - val_mse: 0.0200
Epoch 177/200
1728/1728 [==============================] - 0s 152us/step - loss: 0.0146 - acc: 0.3924 - mae: 0.0500 - msle: 0.0082 - mse: 0.0146 - val_loss: 0.0200 - val_acc: 0.2500 - val_mae: 0.0586 - val_msle: 0.0109 - val_mse: 0.0200
Epoch 178/200
1728/1728 [===========================

In [77]:
np.random.seed(300)
tf.compat.v1.random.set_random_seed(2005)

learn = 0.01
opt = tf.optimizers.Adam(lr= learn, amsgrad = True)
#opt = tf.optimizers.Adadelta(lr= learn)

loss_fctn = 'mean_squared_error'
activ_h = 'tanh'  #selu, tanh
KI_h = 'glorot_uniform'

model = Sequential()
model.add(Dense(100, input_shape=(X_train.shape[1], X_train.shape[2]), activation=activ_h, kernel_initializer=KI_h))
#model.add(Dense(50, input_shape=(X_train.shape[1], X_train.shape[2]), activation=activ_h, kernel_initializer=KI_h))
#model.add(Dense(25, input_shape=(X_train.shape[1], X_train.shape[2]), activation=activ_h, kernel_initializer=KI_h))
model.add(LSTM(32, activation=activ_h, kernel_initializer=KI_h, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]), recurrent_dropout = 0.4,dropout=0.4))
model.add(LSTM(32, activation=activ_h, kernel_initializer=KI_h, return_sequences=True, recurrent_dropout = 0.2,   dropout=0.2))
model.add(LSTM(32, activation=activ_h, kernel_initializer=KI_h, return_sequences=True, recurrent_dropout = 0.2,   dropout=0.2))
model.add(LSTM(32))
model.add(Dense(16, input_dim=32, activation=activ_h, kernel_initializer=KI_h))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss=loss_fctn, optimizer=opt,  metrics=['acc', 'mae', 'msle', 'mse'])
 


NameError: name 'X_train' is not defined